# Regrid RiOMar variables to HEALPix using `regrid_to_healpix` + `xarray.apply_ufunc`

## Prerequisites
1. Run `Create_ROI_from_bbox.ipynb` to generate:
   - `parent_ids.npz` (contains `parent_ids` and `parent_level`)
2. Run `Prep_regrid.ipynb` to generate a temporary input Zarr (e.g. `small.zarr`)
3. Install:
```bash
pip install "git+https://github.com/EOPF-DGGS/regrid_to_healpix.git"
```
## What this notebook does
- Loads the temporary Zarr
- Stacks the source grid into a 1D `point` dimension
- Builds a HEALPix operator once from `nav_lon_*` / `nav_lat_*`
- Applies the transform to selected variables with `xarray.apply_ufunc`
- Writes the HEALPix output to Zarr (e.g. `hp.zarr`)


## Steps
1.	**Setup & load inputs**

Load the temporary Zarr and the ROI definition (parent_ids.npz), and set parameters (levels, chunk sizes, paths).

3.	**Prepare the source grid for apply_ufunc**

Stack the native grid into a 1D point dimension and drop invalid points (NaNs).

4.	**Build the HEALPix operator and regrid**

Build the regridding operator once from nav_lon_* / nav_lat_*, then apply the transform with xarray.apply_ufunc.


5.	**Quick verification / visualization**

Decode and visualize the HEALPix output for sanity checks.



6.	**Align to the ROI and write output Zarr**

Expand parent_ids to child_level, reindex to the target cell_ids, chunk, and write the aligned HEALPix dataset to Zarr.

## Output dataset conventions
- `cell_ids` coordinate (nested indexing)
- `cell_ids` attributes: grid name, level, indexing scheme, ellipsoid (WGS84)

## Next step (big scale)
After validating on the temporary Zarr, switch the input to the full virtual dataset (from the Kerchunk catalog)
and run this notebook as a batch job on HPC. Once verified, publish the resulting Zarr to the FAIR2Adapt endpoint.

## Step 1 — Setup & load inputs

**Goal:** define paths and parameters, then load:
- the temporary input Zarr (from `Prep_regrid.ipynb`)
- the ROI definition `parent_ids.npz` (from `Create_ROI_from_bbox.ipynb`)

In the next cell we:
1. import dependencies
2. set `zarr_file_path`, `zarr_hp_file_path`, `child_level`, `time_chunk_size`
3. load `parent_ids` and `parent_level` from `parent_ids.npz`

In [1]:
import xarray as xr
import numpy as np
import xdggs
import healpix_geo

zarr_file_path= '/Users/todaka/data/RIOMAR/small.zarr'
time_chunk_size = 24  # 1 day as a chunk
child_level=13
zarr_hp_file_path = '/Users/todaka/data/RIOMAR/small_hp.zarr'
with np.load("parent_ids.npz") as data:
    parent_ids = data["parent_ids"]
    parent_level = int(data["parent_level"])

## Step 2 — Prepare the source grid for `apply_ufunc`

**Goal:** convert the 2D native grid into a 1D `point` axis required by the ufunc.

In the next cell we:
1. open the input Zarr
2. `stack(point=("y_rho","x_rho"))`
3. drop invalid points with `dropna(dim="point")`

**Output:** `ds` with a `point` dimension (ready for regridding)

In [2]:
small_ds=xr.open_zarr(zarr_file_path)
ds = small_ds.stack(point=("y_rho","x_rho")).dropna(dim="point")
ds

<xarray.Dataset> Size: 2MB
Dimensions:       (time_counter: 2, s_rho: 40, point: 2699)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * point         (point) object 22kB MultiIndex
    mask          (point) bool 3kB dask.array<chunksize=(2699,), meta=np.ndarray>
    nav_lat_rho   (point) float32 11kB dask.array<chunksize=(2699,), meta=np.ndarray>
    nav_lon_rho   (point) float32 11kB dask.array<chunksize=(2699,), meta=np.ndarray>
  * y_rho         (point) float32 11kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
  * x_rho         (point) float32 11kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Data variables:
    salt          (time_counter, s_rho, point) float32 864kB dask.array<chunksize=(1, 40, 2699), meta=np.ndarray>
    temp          (time_counter, s_rho, point) float32 864kB dask.array<chunksize=(1, 40, 2699), meta=np.ndarray>
    zeta          (time_counter, point) float32 22kB dask.array<chunksize=(1, 2699), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

## Step 3 — Build the HEALPix operator and regrid with `apply_ufunc`

**Goal:** build the HEALPix regridding operator once and apply it to the dataset.

In the next cell we:
1. read `nav_lon_*` / `nav_lat_*` and create the operator (`Set(...)`)
2. compute target `cell_ids` at `child_level`
3. define a 1D transform function `to_healpix_point(data_1d)`
4. apply regridding via `xr.apply_ufunc(...)` to obtain `ds_hp`
5. attach `cell_ids` coordinate and metadata (nested indexing, WGS84)

**Output:** `ds_hp` with dimension `cell_ids`

In [3]:
from regrid_to_healpix.regrid_to_healpix_bilinear import Set


# Build operator once
lon = ds["nav_lon_rho"].values.astype(np.float64)
lat = ds["nav_lat_rho"].values.astype(np.float64)

nr = Set(lon_deg=lon, lat_deg=lat, level=child_level, device="cpu", threshold=0.5, ellipsoid="WGS84")
cell_ids = np.asarray(nr.get_cell_ids(), dtype=np.int64)
ncell = int(cell_ids.size)

def to_healpix_point(data_1d):
    out = nr.transform(np.asarray(data_1d, dtype=np.float64), lam=0.1)
    return np.asarray(out, dtype=np.float64)

# Apply to the whole Dataset: only to chosen data_vars
#vars_to_regrid = ["temp"]  # add "salt", "zeta", ...
ds_in = ds#[vars_to_regrid]

ds_hp = xr.apply_ufunc(
    to_healpix_point,
    ds_in,
    input_core_dims=[["point"]],
    output_core_dims=[["cell_ids"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float64],
    dask_gufunc_kwargs={"output_sizes": {"cell_ids": ncell}},
    keep_attrs=True,  # keep dataset + variable attrs where possible
)

# Re-attach coordinate + its metadata
ds_hp = ds_hp.assign_coords(cell_ids=("cell_ids", cell_ids))
ds_hp["cell_ids"].attrs.update({
    "grid_name": "healpix",
    "level": 13,
    "indexing_scheme": "nested",
    "ellipsoid": "WGS84",
})

ds_hp

/Users/todaka/python/git/regrid_to_healpix/regrid_to_healpix/regrid_to_healpix_bilinear.py:74: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:51.)
  self.M  = M_coo.to_sparse_csr()


<xarray.Dataset> Size: 6MB
Dimensions:       (time_counter: 2, s_rho: 40, cell_ids: 4320)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * cell_ids      (cell_ids) int64 35kB 224374129 224374130 ... 225781262
Data variables:
    salt          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 4320), meta=np.ndarray>
    temp          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 4320), meta=np.ndarray>
    zeta          (time_counter, cell_ids) float64 69kB dask.array<chunksize=(1, 4320), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

## Step 4 — Quick verification / visualization

**Goal:** visually verify that the HEALPix output looks correct.

In the next cell we:
- compute a small result
- decode HEALPix coordinates
- explore on a map for sanity checks

In [4]:
#visualise  and verify
ds_hp.compute().pipe(xdggs.decode).dggs.assign_latlon_coords().dggs.explore(alpha=0.3)

## Step 5 — Align to the ROI and write output Zarr

**Goal:** restrict and align the regridded dataset to the final ROI.

In the next cells we:
1. expand ROI `parent_ids` from `parent_level` to `child_level` (unique target `cell_ids`)
2. reindex `ds_hp` to the target `cell_ids` (fill missing with `NaN`)
3. chunk along `cell_ids` (and time) for writing
4. write the aligned dataset to `zarr_hp_file_path`

**Outputs:**
- aligned HEALPix dataset (`ds_aligned`)
- output Zarr at `zarr_hp_file_path`

In [5]:
# compute the child id from the final interest region
aligned_child_ids = np.unique(healpix_geo.nested.zoom_to(
    parent_ids,
    depth=parent_level,
    new_depth=child_level
))
                              
# Make sure types match (important: your ds_hp cell_ids look like int64)
target_ids = aligned_child_ids.astype(ds_hp["cell_ids"].dtype)
#compute the chunk size 
chunk_size=4**(child_level - parent_level )

# aline the fill non existing values with np.nan, and take out non interestd zone
#
ds_aligned = (
    ds_hp.reindex(cell_ids=target_ids, fill_value=np.nan)
    .chunk({"cell_ids": chunk_size},{"time_counter": time_chunk_size})
)
ds_aligned


<xarray.Dataset> Size: 6MB
Dimensions:       (s_rho: 40, time_counter: 2, cell_ids: 4544)
Coordinates:
  * s_rho         (s_rho) float32 160B -0.9875 -0.9625 ... -0.0375 -0.0125
  * time_counter  (time_counter) datetime64[ns] 16B 2023-12-01T00:48:10 2023-...
    time_instant  (time_counter) datetime64[ns] 16B dask.array<chunksize=(2,), meta=np.ndarray>
  * cell_ids      (cell_ids) int64 36kB 224374592 224374593 ... 225780799
Data variables:
    salt          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 64), meta=np.ndarray>
    temp          (time_counter, s_rho, cell_ids) float64 3MB dask.array<chunksize=(1, 40, 64), meta=np.ndarray>
    zeta          (time_counter, cell_ids) float64 73kB dask.array<chunksize=(1, 64), meta=np.ndarray>
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

In [6]:
%%time

# save the output zarr 
ds_aligned.to_zarr(zarr_hp_file_path,consolidated=True,mode='w')

/Users/todaka/micromamba/envs/pangeo/lib/python3.13/site-packages/zarr/api/asynchronous.py:247: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


CPU times: user 492 ms, sys: 285 ms, total: 776 ms
Wall time: 551 ms


In [7]:
(
    xr.open_dataset(zarr_hp_file_path,consolidated=True,).compute()
    .pipe(xdggs.decode)
    .dggs.assign_latlon_coords()
    .dggs.explore(alpha=0.3)
)

In [8]:
new_ds=ds_hp
display((new_ds.max() - ds.max()).compute(),
(new_ds.min() - ds.min()).compute(),
(new_ds.mean() - ds.mean()).compute())

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B -2.563e-05
    temp     float64 8B -0.01161
    zeta     float64 8B -4.768e-05
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B 0.6243
    temp     float64 8B 0.0
    zeta     float64 8B 0.0
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...

<xarray.Dataset> Size: 24B
Dimensions:  ()
Data variables:
    salt     float64 8B 0.1061
    temp     float64 8B -0.003343
    zeta     float64 8B -0.002013
Attributes: (12/42)
    name:           GAMAR_GLORYS_1h_inst
    description:    Created by xios
    Conventions:    CF-1.6
    title:          GAMAR_GLORYS
    rst_file:       croco_rst.nc
    grd_file:       croco_grd.nc
    ...             ...
    gamma2_expl:    Slipperiness parameter
    x_sponge:       0.0
    v_sponge:       0.0
    sponge_expl:    Sponge parameters : extent (m) & viscosity (m2.s-1)
    SRCS:           main.F step.F read_inp.F timers_roms.F init_scalars.F ini...
    CPP-options:    REGIONAL GAMAR MPI TIDES OBC_WEST OBC_NORTH XIOS USE_CALE...